In [50]:
from torchvision import transforms
from PIL import Image
import torch
from ultralytics import YOLO

In [51]:
model2 = YOLO('./best.pt')
model_vgg = torch.load('./model.pth')

In [52]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize(224),
        transforms.CenterCrop(224),
        
    ]
)

In [53]:
dict_class = {1:'short_weapons',0:'long_weapons',2:'other'}

In [54]:
# results = model2(['./Armed Person Recognition.v2i.yolov8 2/train/images/123_jpg.rf.a532578df87d5f316fcc36b8f0512564.jpg'])
results = model2([r'E:\Downloads\Nelson11.jpg'])


0: 640x512 2 man_with_weapons, 1 man_without_weapon, 3 weapons, 81.7ms
Speed: 3.0ms preprocess, 81.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 512)


In [55]:
for r in results:
    r.names = {2: 'man_with_weapon', 1: 'man_without_weapon', 0: 'weapon',
               3: 'long_weapons', 4: 'short_weapons', 5: 'other'}
    img = Image.fromarray(r.orig_img[..., ::-1])

    for i, bord in enumerate(r.boxes):
        if bord.cls == 'weapon' or bord.cls == 0:

            data = transform(img.crop((
                             bord.xyxy[0][0].item(),
                             bord.xyxy[0][1].item(),
                             bord.xyxy[0][2].item(),
                             bord.xyxy[0][3].item()
                             )))
            data = torch.reshape(data, (1, *data.shape))
            result = model_vgg(data)
            label = str(dict_class[torch.max(result.data, 1).indices.item()])
            print(label)
            try:
                r.boxes.cls[i] = 3+torch.max(result.data, 1).indices
            except RuntimeError:
                print(i, 3+torch.max(result.data, 1).indices, r.boxes.cls)

    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])
    im.show()  # show image
    # im.save('results.jpg')  # save image

long_weapons
2 tensor([3]) tensor([2., 2., 3., 2., 1., 0.], device='cuda:0')
short_weapons
5 tensor([5]) tensor([2., 2., 3., 2., 1., 5.], device='cuda:0')
